### Asset Category Management Tutorial

This Jupyter Notebook provides a step-by-step guide on how to manage `AssetCategory` using the `mainsequence.vam_client` library. We will cover:

1. **Fetching Specific Assets for Different Execution Venues** (Binance for Crypto & Alpaca for US Stocks)
2. **Creating Asset Categories**
3. **Reading Existing Categories**
4. **Updating Categories by Adding More Assets**
5. **Deleting Categories**


In [1]:
# Import necessary modules
import dotenv
import os
dotenv.load_dotenv('../../../.env')

import mainsequence.client
from mainsequence.client import Asset, AssetCategory
from mainsequence.client import CONSTANTS



2025-05-12T07:11:15.708013Z [debug    ] Getting Auth Headers ASSETS_ORM application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:39 in refresh_headers())
2025-05-12T07:11:15.986899Z [debug    ] Getting Auth Headers ASSETS_ORM application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:39 in refresh_headers())
2025-05-12T07:11:16.370651Z [debug    ] took 0.3704 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/pods/projects/get_user_default_project/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())
2025-05-12T07:11:16.371782Z [info     ] Set remote data source to orm_class='DataSource' id=1 display_name='Default TS GCP' organization=1 class_type='timescale_db' status='AVAILABLE' extra_arguments=None application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at models_tdag.py:2101 in set_remote_db())


## Step 1: Fetch Specific Assets

In [2]:
# Select specific crypto assets from Binance
crypto_symbols = ["BTCUSDT", "ETHUSDT", "SOLUSDT"]
binance_crypto_assets = Asset.filter(ticker__in=crypto_symbols, execution_venue__symbol=CONSTANTS.BINANCE_EV_SYMBOL)

# Select specific US stock assets from Alpaca
stock_symbols = ["AAPL", "TSLA", "MSFT"]
alpaca_us_stocks =Asset.filter(ticker__in=stock_symbols, execution_venue__symbol=CONSTANTS.ALPACA_EV_SYMBOL)

# Extract asset IDs
crypto_asset_ids = [asset.id for asset in binance_crypto_assets]
stock_asset_ids = [asset.id for asset in alpaca_us_stocks]

2025-05-12T07:11:21.653616Z [debug    ] took 0.6327 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())
2025-05-12T07:11:22.173699Z [debug    ] took 0.5173 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


## Step 2: Create Asset Categories

In [3]:
from mainsequence.client import User
#get authenticated user to extract its organization and query if the categories already exists
user=User.get_authenticated_user_details()
org_uid = user.organization.uid  # 

2025-05-12T07:12:19.694478Z [debug    ] took 0.4458 seconds. Requesting GET from http://127.0.0.1:8000/user/api/user/get_user_details/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


In [9]:
# Check if the "Favorite Crypto" category exists
crypto_asset_ids = []
crypto_category = AssetCategory.filter(organization_owner_uid=org_uid, display_name="Favorite Crypto")
if len(crypto_category)==0:
    crypto_category = AssetCategory.create(display_name="Favorite Crypto", source="user_defined", assets=crypto_asset_ids)
    print(f"Created Categories: Crypto: {crypto_category}")
else:
    crypto_category=crypto_category[0]

2025-05-12T07:17:32.327107Z [debug    ] took 0.3848 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset-category/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())
2025-05-12T07:17:32.808222Z [debug    ] took 0.4798 seconds. Requesting POST from http://127.0.0.1:8000/orm/api/assets/asset-category/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Created Categories: Crypto: orm_class='AssetCategory' id=38 unique_identifier='favorite_crypto' display_name='Favorite Crypto' source='user_defined' assets=[] organization_owner_uid='00000000-0000-0000-0000-000000000001' description=None


In [15]:

# Check if the "Favorite US Stocks" category exists
stocks_category = AssetCategory.filter(organization_owner_uid=org_uid, name="Favorite US Stocks")
if len(stocks_category)==0:
    stocks_category = AssetCategory.create(name="Favorite US Stocks", source="user_defined", assets=stock_asset_ids)
    print(f"Created Categories: Stocks: {stocks_category}")
else:
    stocks_category=stocks_category[0]
    


2025-05-12T07:22:10.494406Z [debug    ] took 0.9777 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset-category/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


## Step 3: Read Existing Categories

In [11]:
existing_categories = AssetCategory.filter()
print(f"Existing Categories: {existing_categories}")

2025-05-12T07:17:46.125630Z [debug    ] took 0.9929 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset-category/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Existing Categories: [Top 100 Crypto Market Cap source: coingecko, 100 assets, Top 10 Crypto Market Cap source: coingecko, 10 assets, Top 50 Crypto Market Cap source: coingecko, 50 assets, MAGNIFICENT 7 source: mainsequence, 7 assets, S&P500 Constitutents source: datahub.io, 472 assets, Top 10 US Equity Market Cap source: polygon, 10 assets, Top 50 US Equity Market Cap source: polygon, 50 assets, Top 100 US Equity Market Cap source: polygon, 100 assets, Favorite Crypto source: user_defined, 0 assets]


## Step 4: Update Crypto Category by Adding More Assets

In [12]:
# Fetch additional assets (for example, adding ADAUSDT)
new_crypto_asset = Asset.filter(ticker="ADAUSDT", execution_venue__symbol=CONSTANTS.BINANCE_EV_SYMBOL)

# Extend existing category with new asset
crypto_category=crypto_category.append_assets(asset_ids=[new_crypto_asset[0].id])


print(f"Updated Crypto Category: {crypto_category}")
print(crypto_category.assets)

2025-05-12T07:17:50.501737Z [debug    ] took 0.5160 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())
2025-05-12T07:17:51.112815Z [debug    ] took 0.6097 seconds. Requesting POST from http://127.0.0.1:8000/orm/api/assets/asset-category/38/append-assets/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Updated Crypto Category: orm_class='AssetCategory' id=38 unique_identifier='favorite_crypto' display_name='Favorite Crypto' source='user_defined' assets=[23812] organization_owner_uid='00000000-0000-0000-0000-000000000001' description=None
[23812]


## Step 5: Delete a Category

In [16]:
stocks_category.delete()
print(f"Deleted category: Favorite US Stocks")

2025-05-12T07:22:14.184672Z [debug    ] took 0.4690 seconds. Requesting DELETE from http://127.0.0.1:8000/orm/api/assets/asset-category/33/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Deleted category: Favorite US Stocks


### Summary
- We selected specific assets from Binance (crypto) and Alpaca (stocks).
- We created asset categories for them.
- We read and updated categories.
- Finally, we deleted a category when it was no longer needed.

This tutorial helps you manage asset categories efficiently using `mainsequence.vam_client`. 🚀